# Semantic Search

In [1]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

/home/migueldcarvalho/miniforge3/envs/DataScience/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
encoder = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B", device='cpu')

In [4]:
from googletrans import Translator

translator = Translator()

In [5]:
clientd = QdrantClient(path="../metadados/VectorStore/") # Carrega vectorstore em disco

In [9]:
'''

query_points: metodo para fazer buscas sobre coleção(vectorstore)


Testes preliminares:
    
    Utilizar query em inglês pode trazer resultados mais relevantes, mesmo para buscas em português.
    
    Descrever o papel do modelo na instrução (Instrução:) melhora significativamente a relevância dos resultados.

Futuros caminhos de pesquisa:

    Estudar tasks mais elaboradas

    Talvez utlizar tasks dinamicas, ou seja, descrever o papel do modelo com base na query do usuário.

    Utlizar na vectorestore dados como titulo, nome e descrição para melhorar a relevância dos resultados, algumas descrições não possuem
    nome do nome do catalogo o que leva a uma busca menos precisa.

    Encontrar um meio de filtrar dados do catalogo por Datas, tags, formatos, temas(saúde, educação, segurança pública)...

    Talves dispensar o catalogo e fazer uma vector store unica ou fazer duas vectorstores separadas?

        Caso haja duas vectorstores, como conectar ambas? 
        A vectorestore dentro do catalogo pode ser feita no momento da query?
'''

tasks = ["Você é um motor de busca, devolva dados mais relevantes",
         "Você é um motor de busca, devolva dados mais relevantes com base na busca"]

queries = ['quero dados de criminalidade',
           "Dados do bolsa família",
           "Quero informsções sobre infecções hospitalares",
           "numero de Casamentos"]

query_pt = f"Instrução: {tasks[0]} Query: {queries[0]}"
query_en = await translator.translate(query_pt, src='pt', dest='en')

query = query_en.text

print("Query em português:", query_pt)
print("Query utilizada:", query)

hits = clientd.query_points(
    collection_name="Catalogo_metadados",
    query=encoder.encode(query).tolist(),
    limit=200,
).points

print(hits)

for hit in hits:
    print("score:", hit.score)
    print("Titulo: ",hit.payload.get('title'))
    print("Nome: ", hit.payload.get('nome'))
    print("Descrição: ", hit.payload.get('descricao'))
    print('\n')

Query em português: Instrução: Você é um motor de busca, devolva dados mais relevantes Query: quero dados de criminalidade
Query utilizada: Instruction: You are a search engine, return more relevant data Query: I want crime data
[ScoredPoint(id='42e2320b-ea67-4fdc-896f-71363e043fc6', version=0, score=0.5977224095277816, payload={'id': '42e2320b-ea67-4fdc-896f-71363e043fc6', 'title': 'RENAEST', 'nome': 'renaest', 'descricao': 'Quantitativo de sinistros de trânsito por estado, por tipo, por tipo de via (federal, estadual, municipal, distrital) entre outras classificações. Sistema RENAEST\r\n\r\n* Atualização mensal.\r\n* Formato ZIP.', 'catalogacao': '22/07/2024 20:26:50', 'nomeOrganizacao': 'ministerio-dos-transportes', 'ultimaAlteracaoMetadados': '14/10/2025 19:12:41', 'ultimaAtualizacaoDados': '14/10/2025 19:12:42', 'isAtualizado': False}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='84d4489d-edb6-4b87-97d7-fe48d410c684', version=0, score=0.590052639655803, payload=